# First MSSP example

In [1]:
import gurobipy as gb
from gurobipy import GRB
import pandas as pd
import numpy as np
import math
from collections import namedtuple

## Manage datas

Read the synthetic instance of Section 3.2

In [2]:
nodes_number = 25
agents_number = 4
synthetic_5x5_df = pd.read_csv("data/d_it_ij_5x5_1it.csv",
                               index_col=[0,1],
                               decimal=",")

synthetic_5x5_df.head()

it1
1 6  3.0
  7  1.0
2 6  1.0
  7  1.0
  8  1.0

Set up datas that will be used to solve the problem

In [3]:
def get_nodes(networks_df, fishing_from):

    if fishing_from == "cols":
        starting_nodes = networks_df.columns.get_level_values(0)
        ending_nodes = networks_df.columns.get_level_values(1)
    elif fishing_from == "indx":
        starting_nodes = networks_df.index.get_level_values(0)
        ending_nodes = networks_df.index.get_level_values(1)

    return starting_nodes.union(ending_nodes).unique().tolist()

In [4]:
WArc = namedtuple("WArc", ["i", "j", "w"])
w_arcs = [WArc(*arc, weight) for (arc, weight) in synthetic_5x5_df.itertuples()]
idx_of_arc = dict(zip(w_arcs, range(len(w_arcs))))

nodes = get_nodes(synthetic_5x5_df,
                  fishing_from="indx")

agents = [agent for agent in range(1, agents_number+1)]
idx_of_agent = dict(zip(agents, range(len(agents))))

agents_sources = [1, 3, 4, 5]
source_of_agent = dict(zip(agents, agents_sources))

agents_terminus = [21, 23, 24, 25]
terminus_of_agent = dict(zip(agents, agents_terminus))

## Manage the problem

Create the problem

In [5]:
MSPP_pb = gb.Model("First MSPP")
MSPP_pb.setParam("OutputFlag", 0)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-10


Define decision variables

In [6]:
var_shape = len(w_arcs), len(agents)

X = MSPP_pb.addMVar(var_shape,
                    vtype=GRB.BINARY,  # 5) Binary constraints
                    name="k-th agent traverse arc (i,j)")


Define the objective function

In [7]:
# 1-3) Objective function
MSPP_pb.setObjective(
    gb.quicksum(
        arc.w * X[idx_of_arc[arc], idx_of_agent[k]]
        for arc in w_arcs for k in agents
    ),
    GRB.MINIMIZE
)

Add constraints

In [8]:
# 4) Flow constraints

def compute_flow(X, node, w_arcs, agent):
    flow_out = gb.quicksum(
        X[idx_of_arc[arc], idx_of_agent[agent]]
        for arc in w_arcs if arc.i == node
    )
    flow_in = gb.quicksum(
        X[idx_of_arc[arc], idx_of_agent[agent]]
        for arc in w_arcs if arc.j == node
    )
    return flow_out - flow_in


for k in agents:
    for i in nodes:
        if i == source_of_agent[k]:
            MSPP_pb.addConstr(compute_flow(X, i, w_arcs, k) == 1,
                              name=f"Flow constr related to agent {k} in node {i}")
        elif i == terminus_of_agent[k]:
            MSPP_pb.addConstr(compute_flow(X, i, w_arcs, k) == -1,
                              name=f"Flow constr related to agent {k} in node {i}")
        else:
            MSPP_pb.addConstr(compute_flow(X, i, w_arcs, k) == 0,
                              name=f"Flow constr related to agent {k} in node {i}")


Solve the problem

In [9]:
MSPP_pb.optimize()

Report results

In [10]:
print("Result of the optimization is:")
if MSPP_pb.Status == 2:
    print("optimal")
elif MSPP_pb.Status == 3:
    print("infeasible")
elif MSPP_pb.Status == 5:
    print("unbounded")
else:
    print("Some other return status")

Result of the optimization is:
optimal


In [11]:
print(f"Minimum distance covered is {MSPP_pb.ObjVal}")

Minimum distance covered is 16.0


In [12]:
for k in agents:
    print(f"Agent {k} will follow the path:")
    for arc in w_arcs:
        if math.isclose(X.x[idx_of_arc[arc], idx_of_agent[k]], 1):
            print(f"{arc.i}->{arc.j}", end="\t")
    print()


Agent 1 will follow the path:
1->7	7->13	13->17	17->21	
Agent 2 will follow the path:
3->8	8->13	13->18	18->23	
Agent 3 will follow the path:
4->8	8->13	13->18	18->24	
Agent 4 will follow the path:
5->9	9->15	15->19	19->25	
